# **Traffic Incidents optimization**

## **Context**

In some high-density traffic cities like Bogotá, slow attention to traffic incidents remains an issue, and a method to deal with it is yet to be implemented. Quick response is important for most drivers. Those involved in accidents would have higher chances of survival, and even those never involved in an incident would see the benefit of faster moving traffic.

Let us define what we mean by traffic incident or accident. Firstly, an accident implies the happening of a crash, which could be one of a car against an object, or between vehicles. On the other hand, incident is a more general term, it refers to any event that negatively affects traffic flow, so in those terms, an accident is a specific kind of incident.

In 2019, Bogotá had the alarming rate of a traffic accident every five to six minutes, and in 2018 there were a total of 50 800 traffic incidents, a number excessively high, that if it were to be reduced, a great quantity of time would be saved. This of course puts interest on projects trying to fix this problem, which can take many different approaches and focal points. Among these are 



## **Objectives**

## **Model formulation**

El modelo para la solución a nuestro problema se divide en 2 partes, la primera parte  se encarga de  maximizar los pesos totales  de la gravedad de los incidentes que se abordarán con los recursos de respuesta actualmente  disponible, y la  segunda parte se encarga de minimizar el tiempo total de viaje de la respuesta de todas las unidades.

$\textbf{Fase 1:}$
La función objetivo en esta fase es: 

$(1a)$ $$\max \sum_{j=1}^{N_i}a_{s(j)}z_j$$

bajo las restricciones:

$(1b)$ $$\sum_{i=1}^{N_t} x_{ij}\geq b_{s(j)} z_j,\hspace{1cm} \forall j\in N_i$$ 

$(1c)$  $$\sum_{i=1}^{N_i} x_{ij}\leq 1,\hspace{1cm} \forall i\in RU $$ 

$(1d)$  $$x_{ij}=0 \text{ o } 1, \hspace{1cm} \forall i\in RU, \forall j\in N_i$$

$(1e)$   $$z_{j}=0 \text{ o } 1, \hspace{1cm}, \forall j\in N_i$$

$\textbf{Fase 2: }$ La función objetivo en esta parte es:

$(2a)$ $$\min \sum_{i=1}^{N_t}\sum_{j=1}^{N_i} x_{ij}t_{ij}(t)$$

bajo las restricciones: 

$(2b)$ $$\sum_{i=1}^{N_t} x_{ij}\geq b_{s(j)} z_j,\hspace{1cm} \forall j\in N_i$$ 

$(2c)$  $$\sum_{i=1}^{N_i} x_{ij}\leq 1,\hspace{1cm} \forall i\in RU $$ 

$(2d)$  $$x_{ij}=0 \text{ o } 1, \hspace{1cm} \forall i\in RU, \forall j\in N_i$$


Donde 

$t_{ij}(t): $ tiempo de viaje del camino más corte entre los puntos $i$, $j$ en el paso de tiempo $t$

$S(j): $ La función que devuelve la prioridad de un incidente dado $j$ (este es un número entre 1 y 4, donde 1 es la menor prioridad)

$a_l: $ El peso de la gravedad de los incidentes con l-ésima prioridad

$b_l:$ El número de grúas necesarias para ser despachadas a un incidente con l-ésima prioridad(se necesitan 2 para una prioridad de 4, 1 para una prioridad de 3, 1 para una prioridad de 2, y 0 para una prioridad de 1)

$N_i:$ El número de incidentes que están esperando grúas 

$N_t:$ El número de grúas en todo en sistema

$RU: $ Unidades de respuesta 


Variables de decisión

$z_j=1$  Si el incidente j-ésimo será tratado por ciertas grúas, de lo contrario 0

$x_{ij}=1$ Si la grúa i se despacha al incidente j, de lo contrario 0 

En la fase 1 del modelo, la función objetivo (1a) es maximizar los pesos totales de la gravedad de los incidentes a tratar
sin exceder la limitación de número de unidades de respuesta  disponibles. La restricción (1b) garantiza que se envíe una cantidad suficiente
de grúas a cada lugar del incidente. La restricción (1c) indica que una grúa solo puede atender un solo incidente en un
momento dado. Las restricciones (1d) y (1e) representan restricciones de enteros, la ecuación (2a) se encarga de minimizar el tipo de viaje estimado de las unidades que se requieren para moverse desde sus ubicaciones actuales a las ubicaciones de los incidentes asignados.

Este problema pertenece a la programación lineal entera (puesto que la función objetivo es lineal, tanto en la fase 1 como en la fase 2 es un producto punto entre 2 vectores, y las restricciones son lineales y las variables de decisión son entera), dado que este problema en el fondo es lineal tiene solución por medio de algoritmos que involucren convexidad, pero por las restricciones enteras resulta que este problema no siempre va a tener solución de hecho es un problema NP-completo.

## **Model implementation**

Imports

In [ ]:
import cvxpy as cp
import numpy as np
import pandas as pd
# np.set_printoptions(precision=3, suppress=True)

First step is to maximize the severities of the incidents under current response resources(Number of available agents)

In [ ]:
t = pd.read_csv("durations.csv").iloc[:,1:]
i = t.shape[0] # number of agents
j = t.shape[1] # number of incidents

b = np.array([np.random.randint(1, 4) for i in range(j)])

In [ ]:
x = cp.Variable((i, j), boolean=True) # agents-incidents matrix
z = cp.Variable(j, boolean=True) # incidents coverage vector

objective = cp.Maximize(b@z) 

constraints = [cp.sum(x, axis=0) >= cp.multiply(b, z), # required agents per incidents
               cp.sum(x, axis=1) <= 1 # one agent covers at max. one incident
               ]
               
problem = cp.Problem(objective, constraints)

problem.solve(verbose=0)

In [ ]:
print("Agents :\n", x.value, end="\n\n")
print("Incidents:\n", z.value)

Next step is to minize the total travel time of all the agents

In [ ]:
objective = cp.Minimize(cp.sum(cp.sum(cp.multiply(x, t), axis=0)))


constraints = [cp.sum(x, axis=0) >= cp.multiply(b, z.value), # required agents per incidents
               cp.sum(x, axis=1) <= 1 # one agent covers at max. one incident
               ]
               
problem = cp.Problem(objective, constraints)

problem.solve()

In [ ]:
np.sum(x.value, axis=1)

## **Model in practice**

In [1]:
import pandas as pd
import json
import plotly.graph_objects as go
import requests
import folium
import plotly.express as px
import matplotlib.pyplot as plt

In [23]:
incidentsdf = pd.read_csv('incidents_may.csv', usecols=["id", "priority", "implicated", "incident_time", "latitude", "longitude"])

# drop duplicates
incidentsdf = incidentsdf.drop_duplicates(subset="id", keep="first")

# cast incident_time to datetime
incidentsdf["incident_time"] = pd.to_datetime(incidentsdf["incident_time"])

# fill nan to 0 and cast implicated to int
incidentsdf["implicated"] = incidentsdf["implicated"].fillna(0)
incidentsdf["implicated"] = incidentsdf["implicated"].astype(int)

# calculate intervals and create column
new = incidentsdf.groupby(pd.Grouper(key="incident_time", freq='1H')).apply(lambda x: x['incident_time'])
incidentsdf["interval"] = new.index.get_level_values(0)

# multiindex by interval, then id
incidentsdf = incidentsdf.set_index(["interval", "id"])

# print head
incidentsdf.head()

priority  latitude  longitude  implicated  \
interval            id                                                 
2022-04-30 23:00:00 57518         2  4.705279 -74.127686           2   
                    57519         1  4.600571 -74.143379           3   
2022-05-01 00:00:00 57520         1  4.647777 -74.137123           1   
                    57521         1  4.720405 -74.075302           2   
                    57522         1  4.614258 -74.167252           2   

                                incident_time  
interval            id                         
2022-04-30 23:00:00 57518 2022-04-30 23:59:00  
                    57519 2022-04-30 23:54:00  
2022-05-01 00:00:00 57520 2022-05-01 00:08:00  
                    57521 2022-05-01 00:10:00  
                    57522 2022-05-01 00:11:00

In [15]:
# number of incindents per interval
incidentsdf.groupby(["interval"]).size().sort_values(ascending=False)

interval
2022-05-20 18:00:00    50
2022-05-19 18:00:00    48
2022-05-24 17:00:00    48
2022-05-23 10:00:00    46
2022-05-20 20:00:00    44
                       ..
2022-05-10 02:00:00     1
2022-05-23 02:00:00     1
2022-05-12 01:00:00     1
2022-05-11 04:00:00     1
2022-05-06 00:00:00     1
Length: 592, dtype: int64

In [24]:
agentsdf = pd.read_csv('agents_may.csv', usecols=["dev_id", "time_stamp", "latitude", "longitude"])

# cast time_stamp to datetime
agentsdf["time_stamp"] = pd.to_datetime(agentsdf["time_stamp"])

# calculate intervals and create column
new = agentsdf.groupby(pd.Grouper(key="time_stamp", freq='1H')).apply(lambda x: x['time_stamp'])
agentsdf["interval"] = new.index.get_level_values(0)

# multiindex by interval, then id
agentsdf = agentsdf.set_index(["interval", "dev_id"]).reset_index().drop_duplicates(subset=["interval", "dev_id"], keep="last").set_index(["interval", "dev_id"])

# print head
agentsdf.head()

time_stamp  latitude  longitude
interval   dev_id                                                  
2022-05-01 868033050102534 2022-05-01 00:01:42  4.650030 -74.147322
           868033050090085 2022-05-01 00:01:47  4.581688 -74.128348
           868033050103508 2022-05-01 00:01:52  4.551098 -74.147635
           868033050101650 2022-05-01 00:01:55  4.619840 -74.159668
           868033050102989 2022-05-01 00:01:56  4.651116 -74.136658

In [25]:
# format incident locations according to the routing API
incidents_locations = incidentsdf[["longitude","latitude"]].apply(lambda x: ','.join(x.values.astype(str)), axis=1)
incidents_locations.head()

interval             id   
2022-04-30 23:00:00  57518       -74.127685546875,4.705278873443604
                     57519    -74.14337921142578,4.6005706787109375
2022-05-01 00:00:00  57520    -74.13712310791016,4.6477766036987305
                     57521     -74.07530212402344,4.720405101776123
                     57522     -74.16725158691406,4.614258289337158
dtype: object

In [26]:
# format agent locations according to the routing API
agents_locations = agentsdf[["longitude","latitude"]].apply(lambda x: ','.join(x.values.astype(str)), axis=1)
agents_locations

interval             dev_id         
2022-05-01 00:00:00  868033050102534    -74.1473218,4.6500299
                     868033050090085    -74.1283484,4.5816876
                     868033050103508    -74.1476347,4.5510981
                     868033050101650      -74.1596683,4.61984
                     868033050102989    -74.1366581,4.6511155
                                                ...          
2022-05-30 23:00:00  868033050100140    -74.1179133,4.5588983
                     868033050099250    -74.1772431,4.6360906
                     868033050099821    -74.0850125,4.5963301
                     868033050090317      -74.15399,4.6363333
                     868033050101932     -74.1399241,4.617186
Length: 86101, dtype: object

In [19]:
incidents_sample = incidents_locations.sample(1000, random_state=0)
agents_sample = agents_locations.sample(1000, random_state=0)

In [20]:
incidents_sample = incidents_locations.xs("2022-05-01 01:00:00")
agents_sample = agents_locations.xs("2022-05-01 01:00:00")

In [27]:
incidentsdf[incidentsdf["incident_time"].dt.to_period("d") == "2022-05-15"]

priority  latitude  longitude  implicated  \
interval            id                                                 
2022-05-15 00:00:00 63576         2  4.711182 -74.125687           2   
                    63577         2  4.668450 -74.120399           2   
                    63578         0  4.615568 -74.187256           1   
                    63579         1  4.571099 -74.089355           2   
                    63580         1  4.615561 -74.161598           2   
...                             ...       ...        ...         ...   
2022-05-15 22:00:00 63904         2  4.679541 -74.109406           0   
2022-05-15 23:00:00 63905         2  4.569134 -74.186516           2   
                    63906         1  4.665658 -74.130547           4   
                    63907         2  4.725343 -74.088745           0   
                    63908         1  4.557262 -74.138222           2   

                                incident_time  
interval            id                         
2022-05-15 00:00:00 63576 2022-05-15 00:03:00  
                    63577 2022-05-15 00:03:00  
                    63578 2022-05-15 15:05:00  
                    63579 2022-05-15 00:08:00  
                    63580 2022-05-15 00:12:00  
...                                       ...  
2022-05-15 22:00:00 63904 2022-05-15 23:22:00  
2022-05-15 23:00:00 63905 2022-05-15 23:26:00  
                    63906 2022-05-15 23:27:00  
                    63907 2022-05-15 23:40:00  
                    63908 2022-05-15 00:50:00  

[333 rows x 5 columns]

In [21]:
# select a subset of one day to display
incidentsdf_subset = incidentsdf[incidentsdf["incident_time"].dt.to_period("d") == "2022-05-15"].reset_index().copy()
agentsdf_subset = agentsdf[agentsdf["time_stamp"].dt.to_period("d") == "2022-05-15"].reset_index().copy()
# add hour colums for slider purposes
incidentsdf_subset["hour"] = incidentsdf_subset["incident_time"].dt.hour.astype(str)
agentsdf_subset["hour"] = agentsdf_subset["time_stamp"].dt.hour.astype(str)

In [22]:
incidentsdf_subset

,interval,id,priority,latitude,longitude,implicated,incident_time,hour
0,2022-05-15 00:00:00,63576,2,4.711182,-74.125687,2,2022-05-15 00:03:00,0
1,2022-05-15 00:00:00,63577,2,4.668450,-74.120399,2,2022-05-15 00:03:00,0
2,2022-05-15 00:00:00,63578,0,4.615568,-74.187256,1,2022-05-15 15:05:00,15
3,2022-05-15 00:00:00,63579,1,4.571099,-74.089355,2,2022-05-15 00:08:00,0
4,2022-05-15 00:00:00,63580,1,4.615561,-74.161598,2,2022-05-15 00:12:00,0
...,...,...,...,...,...,...,...,...
328,2022-05-15 22:00:00,63904,2,4.679541,-74.109406,0,2022-05-15 23:22:00,23
329,2022-05-15 23:00:00,63905,2,4.569134,-74.186516,2,2022-05-15 23:26:00,23
330,2022-05-15 23:00:00,63906,1,4.665658,-74.130547,4,2022-05-15 23:27:00,23
331,2022-05-15 23:00:00,63907,2,4.725343,-74.088745,0,2022-05-15 23:40:00,23


In [13]:
hour = 0
agentsdf_subset_hour = agentsdf_subset[agentsdf_subset["hour"] == hour] 
incidentsdf_subset_hour = incidentsdf_subset[incidentsdf_subset["hour"] == hour] 

hours = list(range(24))
map_frames = []

for hour in hours:
    agentsdf_subset_hour = agentsdf_subset[agentsdf_subset["hour"] == hour] 
    incidentsdf_subset_hour = incidentsdf_subset[incidentsdf_subset["hour"] == hour] 
    map_frames.append(go.Frame(data=[go.Scattermapbox(
                                        lat=agentsdf_subset_hour["latitude"],
                                        lon=agentsdf_subset_hour["longitude"],
                                        mode='markers',
                                        marker=go.scattermapbox.Marker(
                                                size=8,
                                                color='rgb(255, 0, 0)',
                                                opacity=0.7),
                                        name="Agents"), 
                                    go.Scattermapbox(
                                        lat=incidentsdf_subset_hour["latitude"],
                                        lon=incidentsdf_subset_hour["longitude"],
                                        mode='markers',
                                        marker=go.scattermapbox.Marker(
                                            size=8,
                                            color='rgb(0, 0, 255)',
                                            opacity=0.7),
                                        name="Incidents")]))

hour_steps = []
for hour in hours:
    slider_step = {"args": [
                    [hour],
                    {"frame": {"duration": 300, "redraw": True},
                    "mode": "immediate",
                    "transition": {"duration": 300}}
                    ],
                    "label": str(hour),
                    "method": "update"} 
    hour_steps.append(slider_step)

hour_slider = {
    "active": 0,
    "yanchor": "top",
    "xanchor": "left",
    "currentvalue": {
        "font": {"size": 20},
        "prefix": "Hour: ",
        "visible": True,
        "xanchor": "right"
    },
    "transition": {"duration": 300, "easing": "cubic-in-out"},
    "pad": {"b": 10, "t": 10},
    "len": 0.9,
    "x": 0.1,
    "y": 0,
    "steps": hour_steps
}

fig = go.Figure(data=[go.Scattermapbox(
                            lat=agentsdf_subset_hour["latitude"],
                            lon=agentsdf_subset_hour["longitude"],
                            mode='markers',
                            marker=go.scattermapbox.Marker(
                                size=8,
                                color='rgb(255, 0, 0)',
                                opacity=0.7),
                            name="Agents"), 
                      go.Scattermapbox(
                            lat=incidentsdf_subset_hour["latitude"],
                            lon=incidentsdf_subset_hour["longitude"],
                            mode='markers',
                            marker=go.scattermapbox.Marker(
                                size=8,
                                color='rgb(0, 0, 255)',
                                opacity=0.7),
                            name="Incidents")],
                layout=go.Layout(
                    title="Siniestros e Inicidentes",
                    updatemenus=[{
                        "type":"buttons",
                        "buttons":[{
                            "args": [None, {"frame": {"duration": 500, "redraw": True},
                                            "fromcurrent": True, "transition": {"duration": 300,
                                                                                "easing": "quadratic-in-out"}}],
                            "label": "Play",
                            "method": "animate"
                        },
                        {
                            "args": [[None], {"frame": {"duration": 0, "redraw": True},
                                            "mode": "immediate",
                                            "transition": {"duration": 0}}],
                            "label": "Pause",
                            "method": "animate"
                        }],
                        "direction": "left",
                        "pad": {"r": 10, "t": 87},
                        "showactive": False,
                        "type": "buttons",
                        "x": 0.1,
                        "xanchor": "right",
                        "y": 0,
                        "yanchor": "top"}],
                    autosize=True,
                    hovermode='closest',
                    showlegend=True,
                    mapbox=dict(
                        bearing=0,
                        center=dict(
                            lat=4.624335,
                            lon=-74.063644),
                        pitch=0,
                        zoom=11,
                        style='open-street-map'),
                    height=800,
                    sliders=[hour_slider]),
                frames=map_frames
            )

fig.show()

In [11]:
folium_map = folium.Map(
    location=[4.6534649, -74.0836453], zoom_start=12, tiles="CartoDB positron"
)

incidents_feature_group = folium.FeatureGroup(name="Incidents")
for i in range(len(incidents_sample)):
    marker = folium.CircleMarker(
        location=incidents_sample.iloc[i].split(",")[::-1],
        radius=1,
        color="red",
        fill=True,
        fill_opacity=0.3,
    )
    marker.add_to(incidents_feature_group)

agents_feature_group = folium.FeatureGroup(name="Agents")
for i in range(len(agents_sample)):
    marker = folium.CircleMarker(
        location=agents_sample.iloc[i].split(",")[::-1],
        radius=1,
        color="blue",
        fill=True,
        fill_opacity=0.3,
    )
    marker.add_to(agents_feature_group)

incidents_feature_group.add_to(folium_map)
agents_feature_group.add_to(folium_map)

folium.LayerControl().add_to(folium_map)
folium_map


In [12]:
agents_coordinates = ";".join(agents_sample)
incidents_coordinates = ";".join(incidents_sample)
coordinates = agents_coordinates + ";" + incidents_coordinates

agents_count = agents_sample.shape[0]
sources = ";".join(str(i) for i in range(agents_count))

incidents_count = incidents_sample.shape[0]
destinations = ";".join(
    str(i) for i in range(agents_count, agents_count + incidents_count)
)

#Ojo con correr esto sin estar corriendo el servicio.
durations = json.loads(
    requests.get(
        f"http://127.0.0.1:5000/table/v1/driving/{coordinates}?sources={sources}&destinations={destinations}"
    ).text
)["durations"]


ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=5000): Max retries exceeded with url: /table/v1/driving/-74.0995867,4.615981;-74.1241795,4.6196585;-74.1059283,4.6298267;-74.1366581,4.6511155;-74.1561167,4.5804683;-74.1352133,4.58698;-74.1092917,4.5796633;-74.106445,4.5866167;-74.1161715,4.6694361;-74.116225,4.61031;-74.1547399,4.5883178;-74.140527,4.5983103;-74.1100911,4.5868611;-74.1172057,4.6157602;-74.1354525,4.5763805;-74.1131562,4.6522242;-74.0812621,4.6000139;-74.1045128,4.6822799;-74.108515,4.5516463;-74.1377591,4.5891261;-74.1681975,4.5808575;-74.1028983,4.6184283;-74.11724,4.7138533;-74.1380083,4.5862667;-74.1337581,4.7024305;-74.1167681,4.6204234;-74.1519302,4.6502737;-74.1210833,4.6001233;-74.1353083,4.63264;-74.1217769,4.5574569;-74.0546768,4.6713906;-74.1334068,4.5840831;-74.1970551,4.6411838;-74.1080293,4.6077622;-74.0812033,4.5798129;-74.133665,4.586195;-74.106885,4.615795;-74.0997067,4.6157967;-74.1473483,4.6500037;-74.1945733,4.6230483;-74.1075067,4.6056567;

In [ ]:
#Finally, we can create a dataframe with the durations in seconds.}

df = pd.DataFrame(durations)
#df = df.apply(lambda x: x/60)

#df = df.set_index(agents_sample.index.get_level_values(1))
#df = df.set_axis(list(incidents_sample.index.get_level_values(1)), axis=1)

df


In [ ]:
df.to_csv("durations.csv", index=False)